In [4]:
import glob
import os
import pprint
import traceback

import click
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [5]:
# Extraction function
def tflog2pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    DEFAULT_SIZE_GUIDANCE = {
        "compressedHistograms": 1,
        "images": 1,
        "scalars": 0,  # 0 means load all
        "histograms": 1,
    }
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data

In [6]:
def many_logs2pandas(event_paths):
    all_logs = pd.DataFrame()
    for path in event_paths:
        log = tflog2pandas(path)
        if log is not None:
            if all_logs.shape[0] == 0:
                all_logs = log
            else:
                all_logs = all_logs.append(log, ignore_index=True)
    return all_logs

In [10]:
logdir_or_logfile='/data/tb_logs/crispcityscapes_20221019_163308_hiocnn_tb/crispcityscapes_20221019_163308_hiocnn0_train'

In [11]:
pp = pprint.PrettyPrinter(indent=4)
if os.path.isdir(logdir_or_logfile):
    # Get all event* runs from logging_dir subdirectories
    event_paths = glob.glob(os.path.join(logdir_or_logfile, "event*"))
elif os.path.isfile(logdir_or_logfile):
    event_paths = [logdir_or_logfile]
else:
    raise ValueError(
        "input argument {} has to be a file or a directory".format(
            logdir_or_logfile
        )
    )
event_paths

['/data/tb_logs/crispcityscapes_20221019_163308_hiocnn_tb/crispcityscapes_20221019_163308_hiocnn0_train/events.out.tfevents.1666222646.cityscapescrisp-4w25s-4191966676.30.0']

In [12]:
all_logs = many_logs2pandas(event_paths)